In [16]:
import pandas as pd
import os

In [2]:
df=pd.read_json("dataset/pretrain_hq.jsonl",lines=True)
df.head()

,text
0,<|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文...
1,<|im_start|>根据输入的内容，编写一个类别标签。\n这是一篇介绍如何阅读心电图的文...
2,<|im_start|>客户要求看一份报告，但他没有说明他需要哪份报告。他是指哪份报告？这个...
3,<|im_start|>请从这首诗选出三句话，重新组成另一首新的诗歌。秋天美景尽收眼底，枫叶...
4,<|im_start|>引述一段启发人心的一句话，用于鼓励团队士气。“团队的力量不在于每个人...


In [17]:
df.iloc[0]['text']

'<|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> <|im_start|>帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往会吸引读者的注意力。<|im_end|> <|im_start|>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> <|im_start|>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等。\n文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|>'

In [28]:
def process_and_save_dataset(dataset_list, max_length=512, output_file="dataset/pretrain_hq.jsonl"):
    import json
    
    combined_text = ""
    result_list = []
    
    for data in dataset_list:
        # 添加特殊Token
        processed_text = "<|im_start|>" + data.strip() + "<|im_end|>"
        
        # 如果当前文本加上新样本会超过最大长度
        if len(combined_text + " " + processed_text) > max_length and combined_text:
            # 保存当前拼接的文本
            result_list.append({"text": combined_text+"<|endoftext|>"*10})
            # 重置拼接文本
            combined_text = processed_text
        else:
            # 添加空格分隔符（如果combined_text不为空）
            if combined_text:
                combined_text += " " + processed_text
            else:
                combined_text = processed_text
    
    # 处理最后剩余的文本
    if combined_text:
        result_list.append({"text": combined_text + "<|endoftext|>"*10})
    
    # 以最小IO方式保存到jsonl文件
    with open(output_file, 'a', encoding='utf-8') as f:
        for item in result_list:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    return f"已处理并保存 {len(result_list)} 条数据"

# 使用示例
raw=""
for file in os.listdir():
    if file.endswith(".txt"):
        with open(file, "r", encoding="utf-8") as f:
            raw += f.read()

cleaned_raw = raw.replace(""","").replace(""","").replace("'","").replace("'","").replace("，","").replace("\n","")
dataset = cleaned_raw.split("。")

# 处理并保存数据
process_and_save_dataset(dataset[1:], max_length=512)

'已处理并保存 252 条数据'

## remove

In [27]:
def remove_last_n_entries(jsonl_file, n=362):
    import json
    
    # 读取所有行
    with open(jsonl_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    # 计算要保留的行数
    keep_count = max(0, len(lines) - n)
    remaining_lines = lines[:keep_count]
    
    # 重写文件
    with open(jsonl_file, 'w', encoding='utf-8') as f:
        f.writelines(remaining_lines)
    
    return f"已从 {jsonl_file} 中删除最后 {n} 条数据，剩余 {keep_count} 条"

# 使用示例
remove_last_n_entries("dataset/pretrain_hq.jsonl", 362)

'已从 dataset/pretrain_hq.jsonl 中删除最后 362 条数据，剩余 1413259 条'

## 添加结束符

In [29]:
def add_endoftext_to_jsonl(jsonl_file, batch_size=1000):
    import json
    import os
    import tempfile
    
    # 创建临时文件
    temp_file = tempfile.NamedTemporaryFile(mode='w', encoding='utf-8', delete=False)
    temp_filename = temp_file.name
    
    endoftext_str = "<|endoftext|>" * 10  # 重复150次
    
    # 记录处理的行数和修改的行数
    processed = 0
    modified = 0
    
    try:
        # 逐批次处理文件
        with open(jsonl_file, 'r', encoding='utf-8') as in_file:
            batch = []
            for line in in_file:
                processed += 1
                try:
                    data = json.loads(line.strip())
                    # 检查是否以<|endoftext|>结尾
                    if "text" in data and not data["text"].endswith("<|endoftext|>"):
                        data["text"] += endoftext_str
                        modified += 1
                    batch.append(json.dumps(data, ensure_ascii=False) + '\n')
                    
                    # 批处理写入
                    if len(batch) >= batch_size:
                        temp_file.writelines(batch)
                        batch = []
                except json.JSONDecodeError:
                    # 处理无效JSON的情况
                    temp_file.write(line)
            
            # 写入最后一批
            if batch:
                temp_file.writelines(batch)
        
        # 关闭临时文件
        temp_file.close()
        
        # 替换原始文件
        os.replace(temp_filename, jsonl_file)
        
        return f"已处理 {processed} 条数据，修改了 {modified} 条数据添加结束标记"
    
    except Exception as e:
        # 发生错误时删除临时文件
        if os.path.exists(temp_filename):
            os.unlink(temp_filename)
        return f"处理出错: {str(e)}"

# 使用示例
file_path = "dataset/pretrain_hq.jsonl"
result = add_endoftext_to_jsonl(file_path)
print(result)

已处理 1413511 条数据，修改了 1413511 条数据添加结束标记
